### <font color='#4285f4'>Overview</font>

Overview: This notebook will call the custom Data Analytics Agent in this notebook.  It will use the Cloud Run endpoint to interact with the agent.

Cost:
* Approximate cost: $1 

Author:
* Adam Paternostro

In [ ]:
# Architecture Diagram
from IPython.display import Image
Image(url='https://storage.googleapis.com/data-analytics-golden-demo/colab-diagrams/colab-agent-architecture.png', width=1200)

### <font color='#4285f4'>Video Walkthrough</font>

[Video](https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Demo-Agent-Data-Analytics-Agent.mp4)

In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/data-analytics-golden-demo/colab-videos/Demo-Agent-Data-Analytics-Agent.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

### <font color='#4285f4'>License</font>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### <font color='#4285f4'>PIP Install</font>

In [ ]:
# PIP Installs (if necessary)
import sys

# Commented out for testing, so it speeds things up

# Needed to show the charts returned by conversational analytics
!{sys.executable} -m pip install \
  ijson \
  json-stream \
  vega

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

global_cloud_run_endpoint = "${cloud_run_service_data_analytics_agent_api}"
global_service_account_to_impersonate = "${colab_enterprise_notebook_service_account_email}"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = os.environ["GOOGLE_CLOUD_PROJECT"]
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Configure Data Analytics Agent</font>

##### Current Version

In [ ]:
import json
import html
import markdown
from datetime import datetime
from IPython.display import display, HTML
import requests
import google.auth
import google.auth.transport.requests
import pandas as pd

# --- Icon Definitions (Unchanged) ---
ICONS = {
    'Agent': 'https://storage.googleapis.com/data-analytics-golden-demo/images/ADK-512-color.svg',
    'BigQuery': 'https://storage.googleapis.com/data-analytics-golden-demo/images/BigQuery-32-color.svg',
    'User': 'https://storage.googleapis.com/data-analytics-golden-demo/images/person.svg',
    'Gemini': 'https://storage.googleapis.com/data-analytics-golden-demo/images/Gemini-24-color.svg',
    'VertexAI': 'https://storage.googleapis.com/data-analytics-golden-demo/images/VertexAI-24-color.svg',
    'FactCheck': 'https://storage.googleapis.com/data-analytics-golden-demo/images/fact-check.svg'
}


# --- Dataplex Display Function (Unchanged) ---
def display_catalog_results_as_compact_list(response_data: dict) -> str:
    if not response_data or not response_data.get("results"): return ""
    DATAPLEX_ICONS = {'BIGQUERY': 'https://storage.googleapis.com/data-analytics-golden-demo/images/BigQuery-32-color.svg', 'DATAPLEX': 'https://storage.googleapis.com/data-analytics-golden-demo/images/dataplex.png', 'DEFAULT': 'https://storage.googleapis.com/data-analytics-golden-demo/images/dataplex.png'}
    def format_timestamp(ts):
        if not ts: return "N/A"
        try:
            if ts.endswith('Z'): ts = ts[:-1] + '+00:00'
            return datetime.fromisoformat(ts).strftime('%b %d, %Y, %H:%M:%S UTC')
        except (ValueError, TypeError): return ts
    def to_title_case(s):
        if not s or not isinstance(s, str): return ""
        return html.escape(s).replace('_', ' ').title()
    def parse_entry_type(path: str) -> str:
        if not path or not isinstance(path, str): return "N/A"
        return path.split('/')[-1]
    primary_fields, monospace_fields, list_items_html = {"system", "displayName", "location", "description", "fullyQualifiedName", "entryType"}, {"linkedResource", "fullyQualifiedName", "resource", "name", "parentEntry"}, ""
    for result in response_data.get("results", []):
        dataplex_entry, entry_source = result.get("dataplexEntry", {}), result.get("dataplexEntry", {}).get("entrySource", {})
        if not entry_source: continue
        system, display_name, fqn = entry_source.get("system", "UNKNOWN"), entry_source.get("displayName"), dataplex_entry.get("fullyQualifiedName", "No qualified name")
        card_title, location, description = (display_name if display_name else fqn), entry_source.get("location", "N/A"), entry_source.get("description")
        entry_type, icon_url = parse_entry_type(dataplex_entry.get("entryType")), DATAPLEX_ICONS.get(system.upper(), DATAPLEX_ICONS['DEFAULT'])
        description_html, additional_details_html = (f'<div class="item-description-row">{html.escape(description)}</div>' if description else ""), ""
        all_details, excluded_keys = {**result, **entry_source, **{"entryType_full_path": dataplex_entry.get("entryType")}}, primary_fields | {"dataplexEntry", "snippets"}
        for key, value in sorted(all_details.items()):
            if key in excluded_keys or value is None: continue
            if isinstance(value, (dict, list)): additional_details_html += f"""<div class="info-row details-row complex-value"><strong>{to_title_case(key)}:</strong><pre><code>{html.escape(json.dumps(value, indent=2))}</code></pre></div>"""
            else: additional_details_html += f"""<div class="info-row details-row"><strong>{to_title_case(key)}:</strong><span class="{'monospace-value' if key in monospace_fields else ''}">{html.escape(str(value))}</span></div>"""
        list_items_html += f"""<div class="list-item-card"><details class="collapsible-list-item"><summary class="list-item-summary"><div class="summary-main-content"><div class="summary-title-group"><img src="{icon_url}" alt="{system} icon" class="system-icon"><span class="item-title">{html.escape(card_title)}</span></div>{description_html}</div><div class="summary-aside-meta"><div class="meta-item"><span class="meta-label">System:</span><span class="meta-value">{html.escape(system)}</span></div><div class="meta-item"><span class="meta-label">Type:</span><span class="meta-value">{html.escape(entry_type)}</span></div><div class="meta-item"><span class="meta-label">Location:</span><span class="meta-value">{html.escape(location)}</span></div></div><div class="summary-arrow-group"><svg class="arrow" width="16" height="16" viewBox="0 0 24 24"><path d="M7 10l5 5 5-5" /></svg></div></summary><div class="list-item-content">{additional_details_html}</div></details></div>"""
    return f"""<style>.list-results-container{{font-family:'Roboto',Arial,sans-serif;display:flex;flex-direction:column;gap:8px}}.list-item-card{{background-color:#fff;border:1px solid #dfe1e5;border-radius:8px;overflow:hidden}}.collapsible-list-item summary{{list-style:none}}.collapsible-list-item summary::-webkit-details-marker{{display:none}}.list-item-summary{{display:flex;justify-content:space-between;align-items:flex-start;gap:20px;padding:16px 20px;cursor:pointer;transition:background-color .2s ease}}.list-item-summary:hover{{background-color:#f8f9fa}}.summary-main-content{{display:flex;flex-direction:column;gap:8px;flex-grow:1;min-width:0}}.summary-title-group{{display:flex;align-items:center;gap:15px}}.system-icon{{width:24px;height:24px;flex-shrink:0}}.item-title{{font-size:1.1em;font-weight:500;color:#202124}}.item-description-row{{color:#5f6368;font-size:.9em}}.summary-aside-meta{{display:flex;flex-direction:column;gap:8px;font-size:.85em;color:#5f6368;flex-shrink:0;width:220px}}.meta-item{{display:grid;grid-template-columns:70px 1fr;align-items:baseline}}.meta-label{{font-weight:400}}.meta-value{{color:#202124;font-weight:500;white-space:nowrap;overflow:hidden;text-overflow:ellipsis}}.summary-arrow-group{{padding-top:4px}}.summary-arrow-group .arrow{{transition:transform .2s ease-in-out;fill:none;stroke:#5f6368;stroke-width:2}}.collapsible-list-item[open] > .list-item-summary .arrow{{transform:rotate(180deg)}}.list-item-content{{padding:24px;border-top:1px solid #dfe1e5;background-color:#f8f9fa}}.details-content,.list-item-content{{display:flex;flex-direction:column;gap:12px}}.info-row{{display:grid;grid-template-columns:180px 1fr;gap:10px;align-items:start;font-size:.9em}}.info-row strong{{color:#5f6368;font-weight:500}}.info-row span{{color:#3c4043;word-break:break-all}}.monospace-value{{font-family:'Fira Code','Courier New',monospace}}.details-row.complex-value{{display:block}}.details-row.complex-value strong{{display:block;margin-bottom:8px}}.details-row pre{{margin:0;white-space:pre-wrap;word-break:break-all;background-color:#fff;border:1px solid #dfe1e5;border-radius:4px;padding:10px;font-size:.9em;color:#3c4043}}</style><div class="list-results-container">{list_items_html}</div>"""

# --- BigQuery Display Function (Unchanged) ---
def display_bigquery_rows_as_html(response_data: dict) -> str:
    if not response_data or 'query' not in response_data: return ""
    sql_query, rows = response_data.get('query', 'SQL query not available.'), response_data.get('results', [])
    html_output = f"""<div class="dynamic-table-container"><div class="sql-query-display"><pre><code>{html.escape(sql_query)}</code></pre></div>"""
    if rows:
        headers = rows[0].keys()
        html_output += '<table class="google-table"><thead><tr>'
        for header in headers: html_output += f'<th>{html.escape(str(header))}</th>'
        html_output += '</tr></thead><tbody>'
        for row in rows:
            html_output += '<tr>'
            for header in headers: html_output += f'<td>{html.escape(str(row.get(header, "")))}</td>'
            html_output += '</tr>'
        html_output += '</tbody></table>'
    else: html_output += '<div class="no-data-message">No data returned from query.</div>'
    return html_output + '</div>'

def get_icon_for_author_04(author: str) -> str:
    if "BigQuery" in author: return ICONS['BigQuery']
    if "Coordinator" in author: return ICONS['Agent']
    if "Search" in author: return ICONS['VertexAI']
    return ICONS['Agent']

# --- CENTRAL CONVERSATION HTML GENERATOR (Unchanged) ---
def generate_conversation_html(
    conversation_data,
    show_details: bool = True,
    for_streaming: bool = False,
    is_final_answer: bool = False
) -> tuple[str, str]:
    deferred_output_html, messages_html = "", ""
    arrow_svg = """<svg class="toggle-arrow" width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M7 10L12 15L17 10" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"/></svg>"""
    messages = conversation_data.get('conversation_messages', []) if isinstance(conversation_data, dict) else conversation_data
    for message in messages:
        try: data = json.loads(message.get('message_content', '{}'), strict=False)
        except (json.JSONDecodeError, TypeError): continue
        inner_html_parts = ""
        for part in data.get('content', {}).get('parts', []):
            if 'text' in part and 'thought' not in part:
                chip_html = f'<div class="thought-chip result-chip">Final Answer</div>' if is_final_answer else ''
                markdown_content = markdown.markdown(part['text'], extensions=['nl2br', 'fenced_code'])
                inner_html_parts += f"""<div class="message-bubble result-bubble">{chip_html}<div class="markdown-body">{markdown_content}</div></div>"""
            elif part.get('thought'):
                thought_text_markdown = markdown.markdown(part.get('text', ''), extensions=['nl2br', 'fenced_code'])
                inner_html_parts += f"""<div class="thought-bubble"><div class="thought-chip thought-chip-main">Thought</div><div class="markdown-body">{thought_text_markdown}</div></div>"""
            elif 'functionCall' in part and show_details:
                call, func_name = part['functionCall'], html.escape(part['functionCall'].get('name', ''))
                args_str = json.dumps(call.get('args', {}), indent=2)
                inner_html_parts += f"""<div class="thought-bubble collapsible-container collapsed"><div class="collapsible-header" onclick="toggleCollapse(this)"><div><div class="thought-chip tool-call">Tool Call</div><p><b>Function:</b> {func_name}</p></div>{arrow_svg}</div><div class="collapsible-content"><b>Arguments:</b><pre><code>{html.escape(args_str)}</code></pre></div></div>"""
            elif 'functionResponse' in part:
                response_data = part.get('functionResponse', {}).get('response', {})
                if not response_data or response_data == {"result": None}: continue
                tool_name, placeholder_message = response_data.get("tool_name"), ""
                if tool_name == "run_bigquery_sql":
                    deferred_output_html, placeholder_message = display_bigquery_rows_as_html(response_data), "<p>SQL query executed successfully. The results table will be displayed below the conversation.</p>"
                elif tool_name == "search_data_catalog":
                    deferred_output_html, placeholder_message = display_catalog_results_as_compact_list(response_data), "<p>Dataplex search successful. The results will be displayed below the conversation.</p>"
                if placeholder_message and show_details:
                    inner_html_parts += f"""<div class="thought-bubble collapsible-container collapsed"><div class="collapsible-header" onclick="toggleCollapse(this)"><div><div class="thought-chip tool-response">Tool Output</div></div>{arrow_svg}</div><div class="collapsible-content">{placeholder_message}</div></div>"""
                elif show_details:
                    inner_html_parts += f"""<div class="thought-bubble collapsible-container collapsed"><div class="collapsible-header" onclick="toggleCollapse(this)"><div><div class="thought-chip tool-response">Tool Output</div></div>{arrow_svg}</div><div class="collapsible-content"><pre><code>{html.escape(json.dumps(response_data, indent=2))}</code></pre></div></div>"""
        if inner_html_parts.strip():
            author, icon_url = data.get('author', 'Unknown'), ICONS['FactCheck'] if is_final_answer else get_icon_for_author_04(data.get('author', 'Unknown'))
            messages_html += f"""<div class="message-container agent"><img src="{icon_url}" alt="{author} icon" class="avatar"><div class="message-content">{inner_html_parts}</div></div>"""
    return (messages_html, deferred_output_html) if for_streaming else (messages_html, "")

# --- DataAnalyticsAgent Class (with MODIFIED __init__ and ask methods) ---
class DataAnalyticsAgent:
  def __init__(self, user_id, session_id):
    self.user_id, self.session_id, self.agent_name = user_id, session_id, "data_analytics_agent"
    self.cloud_run_endpoint = global_cloud_run_endpoint
    self.service_account_to_impersonate = global_service_account_to_impersonate
    self.bearer_token, self.bearer_token_timestamp = "", datetime.min
    self.conversation_list, self.conversation_id, self.message_id = [], 0, 0
    self.dataframe = pd.DataFrame() # This is NEW
    if self.get_session(): print("Session already exists.")
    else: self.create_session()

  def get_session(self):
    url = f"{self.cloud_run_endpoint}/apps/{self.agent_name}/users/{self.user_id}/sessions/{self.session_id}"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.get_bearer_token()}"}
    try:
        with requests.get(url, headers=headers) as response: response.raise_for_status(); return "id" in json.loads(response.content)
    except (requests.exceptions.RequestException, json.JSONDecodeError): return False

  def create_session(self):
    url = f"{self.cloud_run_endpoint}/apps/{self.agent_name}/users/{self.user_id}/sessions/{self.session_id}"
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {self.get_bearer_token()}"}
    try:
        with requests.post(url, json={"state": {}}, headers=headers) as r: r.raise_for_status(); print(f"Session created.")
    except Exception as e: print(f"create_session error: {e}")

  def get_bearer_token(self):
    if (datetime.now() - self.bearer_token_timestamp).total_seconds() > 2700:
        print("Getting new identity token...")
        creds, _ = google.auth.default(); creds.refresh(google.auth.transport.requests.Request())
        url = f"https://iamcredentials.googleapis.com/v1/projects/-/serviceAccounts/{self.service_account_to_impersonate}:generateIdToken"
        headers = {"Authorization": f"Bearer {creds.token}", "Content-Type": "application/json"}
        response = requests.post(url, json={"audience": self.cloud_run_endpoint}, headers=headers)
        response.raise_for_status()
        self.bearer_token, self.bearer_token_timestamp = response.json()["token"], datetime.now()
    return self.bearer_token

  def rest_api_sse_endpoint(self, human_message):
    headers = {"Accept": "text/event-stream", "Authorization": f"Bearer {self.get_bearer_token()}"}
    payload = {"appName": self.agent_name, "userId": self.user_id, "sessionId": self.session_id, "newMessage": {"role": "user", "parts": [{"text": human_message}]}, "streaming": True}
    url = f"{self.cloud_run_endpoint}/run_sse"
    try:
        with requests.post(url, headers=headers, json=payload, stream=True) as response:
            response.raise_for_status()
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith('data:'): self.message_id += 1; yield {"message_id": self.message_id, "message_content": line[len('data: '):]}
    except Exception as e: print(f"\nError: {e}"); yield None

  def ask(self, human_input: str, show_details: bool = True) -> None:
    def html_wrapper(content):
        return f"""<html><head><style>body{{background-color:#f8f9fa;font-family:'Roboto',Arial,sans-serif;color:#202124;margin:0;padding:0}}.conversation-container{{width:95%;max-width:1200px;margin:auto}}.message-container{{display:flex;align-items:flex-start;margin-bottom:20px}}.avatar{{width:40px;height:40px;border-radius:50%;margin-right:15px;flex-shrink:0}}.message-content{{display:flex;flex-direction:column;align-items:flex-start;width:100%}}.message-bubble,.thought-bubble{{background-color:#fff;border:1px solid #dfe1e5;border-radius:18px;padding:10px 15px;max-width:95%;box-shadow:0 1px 2px 0 rgba(60,64,67,.3),0 1px 3px 1px rgba(60,64,67,.15);margin-bottom:8px;box-sizing:border-box}}.thought-bubble{{background-color:#3c4043;color:#e8eaed;border:none}}.thought-bubble p,.thought-bubble pre{{margin:5px 0;font-size:.9em}}.thought-bubble .markdown-body p{{color:#e8eaed}}.thought-bubble pre code{{white-space:pre-wrap;word-break:break-all;color:#e8eaed}}.thought-chip{{display:inline-block;padding:3px 10px;font-size:.8em;font-weight:500;border-radius:12px;margin-bottom:8px}}.thought-chip.thought-chip-main{{background-color:#8ab4f8;color:#202124}}.thought-chip.tool-call{{background-color:#fdd663;color:#202124}}.thought-chip.tool-response{{background-color:#81c995;color:#202124}}.thought-chip.result-chip{{background-color:#e8eaed;color:#3c4043}}.markdown-body p,.markdown-body ul{{margin:0 0 8px}}.markdown-body pre{{white-space:pre-wrap;background-color:#f1f3f4;padding:10px;border-radius:4px}}.collapsible-header{{display:flex;justify-content:space-between;align-items:center;cursor:pointer}}.toggle-arrow{{color:#bdc1c6;transition:transform .2s ease-in-out}}.collapsible-content{{max-height:500px;overflow-y:auto;transition:max-height .3s ease;padding-top:10px}}.collapsible-container.collapsed .collapsible-content{{max-height:0;overflow:hidden;padding-top:0}}.collapsible-container.collapsed .toggle-arrow{{transform:rotate(-90deg)}}.dynamic-table-container,.sql-query-display{{margin:10px 0 20px}}.sql-query-display{{background-color:#f1f3f4;padding:15px;border-radius:8px;margin-bottom:15px}}.sql-query-display pre code{{white-space:pre-wrap;word-break:break-all}}.google-table{{border-collapse:collapse;width:100%;border:1px solid #dfe1e5;box-shadow:0 2px 2px rgba(0,0,0,.14)}}.google-table thead tr{{background-color:#4285f4;color:#fff}}.google-table th,.google-table td{{padding:12px 15px;text-align:left;border:1px solid #dfe1e5}}.google-table tbody tr:nth-of-type(even){{background-color:#f8f9fa}}.google-table tbody tr:hover{{background-color:#e8f0fe}}.no-data-message{{padding:15px;text-align:center}}.formatted-result-container{{margin-top:20px;border:1px solid #dfe1e5;border-radius:8px;background-color:#fff;box-shadow:0 1px 2px 0 rgba(60,64,67,.3),0 1px 3px 1px rgba(60,64,67,.15)}}.formatted-result-header{{padding:12px 16px;font-size:1.1em;font-weight:500;color:#202124;border-bottom:1px solid #dfe1e5;background-color:#f8f9fa;border-radius:8px 8px 0 0}}.formatted-result-content{{padding:16px}}</style><script>function toggleCollapse(el){{const c=el.closest('.collapsible-container');c&&c.classList.toggle('collapsed');}}</script></head><body><div class="conversation-container">{content}</div></body></html>"""
    final_deferred_output_html, current_conversation_messages, final_text_content, final_answer_author = "", [], "", "Agent"

    for message in self.rest_api_sse_endpoint(human_input):
        if message is None: break
        current_conversation_messages.append(message)
        try:
            data = json.loads(message.get('message_content', '{}'))
            is_partial = data.get("partial", False)
            is_final_answer_chunk = any('text' in p and 'thought' not in p for p in data.get('content', {}).get('parts', []))
        except (json.JSONDecodeError, TypeError):
            continue

        if is_final_answer_chunk:
            if not is_partial:
                for part in data.get('content', {}).get('parts', []):
                    if 'text' in part:
                        final_text_content = part.get('text', '')
                final_answer_author = data.get('author', 'Agent')
        else:
            message_bubble_html, deferred_output_html = generate_conversation_html([message], show_details=show_details, for_streaming=True)
            if deferred_output_html:
                final_deferred_output_html = deferred_output_html
            if message_bubble_html:
                display(HTML(html_wrapper(message_bubble_html)))

    if final_text_content:
        final_message = {"message_content": json.dumps({"author": final_answer_author, "content": {"parts": [{"text": final_text_content}]}})}
        final_bubble_html, _ = generate_conversation_html([final_message], for_streaming=True, is_final_answer=True)
        if final_bubble_html:
            display(HTML(html_wrapper(final_bubble_html)))

    if final_deferred_output_html:
        result_section_html = f"""<div class="formatted-result-container"><div class="formatted-result-header">Formatted Result</div><div class="formatted-result-content">{final_deferred_output_html}</div></div>"""
        display(HTML(html_wrapper(result_section_html)))

    if current_conversation_messages:
        self.conversation_id += 1
        self.conversation_list.append({"conversation_id": self.conversation_id, "conversation_messages": current_conversation_messages})

    # This is NEW and what I want
    # This new block iterates through the messages from the last interaction,
    # finds the BigQuery tool response, and populates the dataframe.
    for message in current_conversation_messages:
        try:
            data = json.loads(message.get('message_content', '{}'))
            if 'content' not in data or 'parts' not in data.get('content', {}):
                continue
            for part in data.get('content', {}).get('parts', []):
                if 'functionResponse' in part:
                    response_data = part.get('functionResponse', {}).get('response', {})
                    if response_data.get("tool_name") == "run_bigquery_sql":
                        results = response_data.get('results')
                        if results:
                            self.dataframe = pd.DataFrame(results)
                            return
        except (json.JSONDecodeError, TypeError):
            continue

### <font color='#4285f4'>Interact with the Agent</font>

In [ ]:
da_agent = DataAnalyticsAgent("adam","session_00")

#### Ask the agent some basic questions

In [ ]:
da_agent.ask("What tables have customer information")

In [ ]:
da_agent.ask("Count the orders by product category, for the current month. Sum the sales and order count.")

In [ ]:
# We can get the SQL results in a dataframe so we can use in our code
da_agent.dataframe

In [ ]:
# Format the date to "MonthName Year"
from datetime import datetime
formatted_date = datetime.now().strftime("%B %Y")
da_agent.ask(f"Count the number of orders by week and product category for {formatted_date}.", True)

In [ ]:
# This will create a new BigQuery pipeline (you can see it on the left hand side menu)

da_agent.ask("""Create a new BigQuery pipeline named 'Sales Data Pipeline'. 
I want to generate a denormalized sales table within the agentic_beans_curated dataset. 
This table should combine data from the following tables and include the specified fields:
- From order_header: order_header_id, order_header_timestamp, truck_id, customer_id, order_neighborhood, order_header_total, payment_method
- From order_detail: order_detail_id, truck_menu_id, order_quantity, product_id, product_category_id, price, order_detail_total
- From customer: customer_name, customer_email, country_code, customer_total_spend
- From product: product_name, product_description
- From product_category: product_category_name, product_category_description
- From truck: truck_name, truck_license_plate""")

In [ ]:
da_agent.ask("Count the orders by user first name and show the top 10 highest order count from the curated dataset.")

In [ ]:
da_agent.ask("search the data catalog for items that contain 'curated'?")

In [ ]:
# Pass False to hide all the tool outputs and just show the thoughts
da_agent.ask("Search for tables that are releated to customers", False)

In [ ]:
da_agent.ask("List all tables that have Zone of Curated")

In [ ]:
da_agent.ask("""I want to run the following SQL statement:
Count all the orders from the agentic curated dataset where the order date is the current month.
Name the output field "order_count"
""")

In [ ]:
# Have it add the first week
da_agent.ask("What are the order counts for the first week in the year?")

In [ ]:
da_agent.ask("Get me the unique neighborhood from the orders table.")

In [ ]:
da_agent.ask("Now count the orders by neighborhood for the current month")

In [ ]:
da_agent.ask("How many customers are there?")

In [ ]:
da_agent.ask("Why did you choose the customer table in the curated dataset?")

In [ ]:
# Let the agent finish the SQL/English

da_agent.ask(f"""I want to run the following SQL statement:
SELECT COUNT(*) AS order_count FROM `{project_id}.agentic_beans_curated.order_header` WHERE order_header_timestamp is in the current year.
""")

In [ ]:
# Let the agent finish the SQL/English

da_agent.ask(f"""I want to run the following SQL statement:
SELECT count star AS order_count FROM {project_id} agentic beans curated order header WHERE order_header_timestamp is the second week of last year.
Return the week range you used and the order count.
""")

In [ ]:
da_agent.ask("""Create a conversational analytics agent named ca-agent-100.
I want to use the taxi_trips, payment_type, rate_code, trip_type, vendor and location tables.
You can use the nyc_taxi_curated dataset for this.
""")

In [ ]:
da_agent.ask("""Show me the messages that were returned""")